In [ ]:
pip install pandas pyarrow

In [ ]:
import pandas as pd

# Load the Parquet file
parquet_file_path = 'yellow_tripdata_2024-09.parquet'
data = pd.read_parquet(parquet_file_path)

# Convert to CSV
csv_file_path = 'converted.csv'
data.to_csv(csv_file_path, index=False)

print(f'Parquet file {parquet_file_path} has been converted to CSV file {csv_file_path}')


In [4]:
import pandas as pd

# Load the data
df = pd.read_csv('converted.csv') 

# Convert 'tpep_pickup_datetime' to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Extract hour
df['hour'] = df['tpep_pickup_datetime'].dt.hour

# Group by hour and calculate summary statistics
hourly_stats = df.groupby('hour').agg({
    'trip_distance': ['mean', 'std'],
    'fare_amount': ['mean', 'std'],
    'passenger_count': 'count' 
})

# Flatten the multi-index
hourly_stats.columns = ['_'.join(col) for col in hourly_stats.columns]

# Rename columns for clarity
hourly_stats.rename(columns={
    'trip_distance_mean': 'avg_trip_distance',
    'trip_distance_std': 'std_trip_distance',
    'fare_amount_mean': 'avg_fare_amount',
    'fare_amount_std': 'std_fare_amount',
    'passenger_count_count': 'num_trips'
}, inplace=True)

# Reset index to create a DataFrame
hourly_stats = hourly_stats.reset_index()

# Print the reduced dataset
print(hourly_stats)

# Save the reduced dataset to a CSV file
hourly_stats.to_csv('hourly_taxi_stats.csv', index=False)

    hour  avg_trip_distance  std_trip_distance  avg_fare_amount  \
0      0           3.829432           4.810559        19.479443   
1      1           3.264212           4.086253        17.050060   
2      2           2.802090           3.161652        15.312117   
3      3           3.113981           3.606497        16.332604   
4      4           4.477261           5.227941        21.843241   
5      5           6.277956           6.778040        27.809795   
6      6           4.928898           6.053218        22.858763   
7      7           3.780577          19.383750        19.621207   
8      8           3.257333           4.381644        19.064726   
9      9           3.159940           4.803094        18.927381   
10    10           3.197848          15.903750        19.211199   
11    11           3.257084          14.373061        19.875650   
12    12           3.279821           4.532224        20.255782   
13    13           3.562750           4.928625        21.14843

In [16]:
import pandas as pd

# Load the original taxi trip data 
df = pd.read_csv('converted.csv') 

# Convert 'tpep_pickup_datetime' to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Extract hour
df['hour'] = df['tpep_pickup_datetime'].dt.hour

# Extract date
df['date'] = df['tpep_pickup_datetime'].dt.date

# Group by hour and date
hourly_daily_stats = df.groupby(['hour', 'date']).agg({
    'trip_distance': ['mean', 'std'],
    'fare_amount': ['mean', 'std'],
    'passenger_count': 'count' 
})

# Flatten multi-index and rename columns
hourly_daily_stats.columns = ['_'.join(map(str, col)) for col in hourly_daily_stats.columns]
hourly_daily_stats.rename(columns={
    'trip_distance_mean': 'avg_trip_distance',
    'trip_distance_std': 'std_trip_distance',
    'fare_amount_mean': 'avg_fare_amount',
    'fare_amount_std': 'std_fare_amount',
    'passenger_count_count': 'num_trips'
}, inplace=True)

# Calculate total trips per day
daily_trip_count = hourly_daily_stats.groupby('date')['num_trips'].sum()

# Print the total trips per day
print("Total Trips per Day:\n", daily_trip_count)

# Pivot the DataFrame to have hours as columns and dates as rows
pivoted_data = hourly_daily_stats.pivot_table(index='date', columns='hour')

# Flatten the multi-index of the pivoted DataFrame
pivoted_data.columns = ['_'.join(map(str, col)) for col in pivoted_data.columns]

# Reset index to create a regular DataFrame
pivoted_data = pivoted_data.reset_index()

# Print the first few rows of the reduced dataset
print("\nHourly and Daily Statistics:\n", pivoted_data.head())

# Save the reduced dataset to a CSV file
pivoted_data.to_csv('hourly_daily_taxi_stats.csv', index=False)

Total Trips per Day:
 date
2008-12-31         2
2009-01-01         1
2024-08-31        31
2024-09-01     76209
2024-09-02     69557
2024-09-03     95511
2024-09-04    108387
2024-09-05    115881
2024-09-06    111079
2024-09-07    105894
2024-09-08     93413
2024-09-09     97101
2024-09-10    106257
2024-09-11    106904
2024-09-12    120562
2024-09-13    108343
2024-09-14    107278
2024-09-15     91059
2024-09-16     96311
2024-09-17    112977
2024-09-18    118296
2024-09-19    122219
2024-09-20    115231
2024-09-21    112921
2024-09-22     99575
2024-09-23     99121
2024-09-24    102590
2024-09-25    109381
2024-09-26    118179
2024-09-27    110974
2024-09-28    120794
2024-09-29     99415
2024-09-30     58873
2024-10-01         9
Name: num_trips, dtype: int64

Hourly and Daily Statistics:
          date  avg_fare_amount_0  avg_fare_amount_1  avg_fare_amount_2  \
0  2008-12-31                NaN                NaN                NaN   
1  2009-01-01                NaN                Na

PermissionError: [Errno 13] Permission denied: 'hourly_daily_taxi_stats.csv'

In [19]:
import pandas as pd
import numpy as np

# Load your data
df = pd.read_csv("converted.csv")

# Ensure you have a datetime column
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# Extract the hour
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

# Calculate total trips
total_trips = len(df)

sampled_df = pd.DataFrame()
for hour in range(24):
    hour_df = df[df['pickup_hour'] == hour]
    hour_trips = len(hour_df)
    
    if hour_trips > 0: # Avoid division by zero
        sample_size = int(round((hour_trips / total_trips) * 1000))
        
        # Ensure sample size doesn't exceed the number of trips in the hour
        sample_size = min(sample_size, hour_trips)
        
        if sample_size > 0:
            sampled_hour_df = hour_df.sample(n=sample_size, replace=False, random_state=42) # Set random_state for reproducibility
            sampled_df = pd.concat([sampled_df, sampled_hour_df])

# Reset index
sampled_df = sampled_df.reset_index(drop=True)
print(len(sampled_df))
print(sampled_df.head())
# Save the sampled data
sampled_df.to_csv("sampled_taxi_data.csv", index=False)

1001
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2024-09-14 00:30:01   2024-09-14 00:49:09              2.0   
1         2  2024-09-27 00:10:56   2024-09-27 00:30:35              1.0   
2         2  2024-09-01 00:28:56   2024-09-01 00:36:44              2.0   
3         2  2024-09-14 00:50:27   2024-09-14 01:01:49              1.0   
4         2  2024-09-11 00:44:51   2024-09-11 00:51:55              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           1.94         1.0                  N           249           224   
1           2.20         1.0                  N            48           158   
2           1.16         1.0                  N           249           144   
3           3.14         1.0                  N           113           140   
4           0.82         1.0                  N           231           261   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amoun

In [36]:
import pandas as pd
import numpy as np

try:
    # Load your data
    df = pd.read_csv("converted.csv")

    # Convert pickup datetime to datetime objects (handle potential errors)
    try:
        df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    except KeyError:
        print("Error: 'tpep_pickup_datetime' column not found. Check your data.")
        exit()
    except ValueError:
        print("Error: Could not parse 'tpep_pickup_datetime'. Check the format.")
        exit()

    # Extract date and hour
    df['pickup_date'] = df['tpep_pickup_datetime'].dt.date
    df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

    def sample_group(group):
        if not group.empty:
            return group.sample(n=min(1, len(group)), replace=False, random_state=42)  # Sample min 1 and max all
        return pd.DataFrame()  # Return empty DataFrame if group is empty

    # Group by date and hour and apply the sampling function
    sampled_df = df.groupby(['pickup_date', 'pickup_hour']).apply(sample_group).reset_index(drop=True)

    # Ensure all date-hour combinations are present (even if no trips)
    start_date = df['pickup_date'].min()
    end_date = df['pickup_date'].max()
    all_dates = pd.date_range(start=start_date, end=end_date).date

    all_combinations = pd.DataFrame([(date, hour) for date in all_dates for hour in range(24)], columns=['pickup_date', 'pickup_hour'])

    # Merge with all combinations to include missing ones. How='outer' to ensure all rows from both are included
    sampled_df = pd.merge(all_combinations, sampled_df, on=['pickup_date', 'pickup_hour'], how='outer')

    print(f"Number of rows in original data: {len(df)}")
    print(f"Number of rows in sampled data: {len(sampled_df)}")
    print("First few rows of the sampled data:")
    print(sampled_df.head())

    # Save the sampled data
    sampled_df.to_csv("test.csv", index=False)
    print("Sampled data saved to sampled_hourly_daily_taxi_data.csv")

except FileNotFoundError:
    print("Error: 'your_taxi_data.csv' not found. Check the file path.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

C:\Users\prana\AppData\Local\Temp\ipykernel_4272\1355057763.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby(['pickup_date', 'pickup_hour']).apply(sample_group).reset_index(drop=True)


Number of rows in original data: 3110335
Number of rows in sampled data: 138096
First few rows of the sampled data:
  pickup_date  pickup_hour  VendorID tpep_pickup_datetime  \
0  2008-12-31            0       NaN                  NaT   
1  2008-12-31            1       NaN                  NaT   
2  2008-12-31            2       NaN                  NaT   
3  2008-12-31            3       NaN                  NaT   
4  2008-12-31            4       NaN                  NaT   

  tpep_dropoff_datetime  passenger_count  trip_distance  RatecodeID  \
0                   NaN              NaN            NaN         NaN   
1                   NaN              NaN            NaN         NaN   
2                   NaN              NaN            NaN         NaN   
3                   NaN              NaN            NaN         NaN   
4                   NaN              NaN            NaN         NaN   

  store_and_fwd_flag  PULocationID  ...  payment_type  fare_amount  extra  \
0            